In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess, Fourier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
oil=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
sub_ctg=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
holiday_pd=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
stores=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
train_df=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test_df=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
transactions=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

**list of all products.**

In [ ]:
train_df["family"].unique()

**Choosing one product for analysing.**

In [ ]:
aa=train_df
aa['date']=pd.to_datetime(aa['date'])
aa['day']=aa['date'].dt.dayofweek
aa['month']=aa['date'].dt.month
aa['year']=aa['date'].dt.year

**Adding some missing dates and filling oil price with interpolation.**

In [ ]:
df = aa.reset_index()
df = df.sort_values('date')
df['date_f'] = pd.factorize(df['date'])[0]
mapping = dict(zip(df['date_f'], df['date'].dt.date))

idx = pd.date_range(df['date'].head(1).values[0],df['date'].tail(1).values[0])
oil.index = pd.DatetimeIndex(oil['date'])
oil=oil.asfreq('D')
oil['date']=oil.index
oil = oil.drop('2013-12-25 00:00:00')
oil = oil.drop('2014-12-25 00:00:00')
oil = oil.drop('2015-12-25 00:00:00')
oil = oil.drop('2016-12-25 00:00:00')
oil=oil.reset_index(drop=True)
oil['date']=oil['date'].dt.strftime('%Y-%m-%d')

df1_oil=oil.interpolate(limit_direction='Both')
df1_oil['date']=pd.to_datetime(df1_oil['date'])
df1_oil['year']=df1_oil['date'].dt.year

oil['date']=pd.to_datetime(oil['date'])
oil['year']=oil['date'].dt.year

fig, ax = plt.subplots(1,2,figsize=(20,5))


ax[0].set(xlabel='Date', ylabel='Oil price');
ax[1].set(xlabel='Date', ylabel='Oil price');

ax[0].plot(oil['date'],oil['dcoilwtico'],color='0.75')
ax[0].set_title("Oil price - Date")

ax[1].plot(df1_oil['date'],df1_oil['dcoilwtico'],color='0.25')
ax[1].set_title("Oil price - Date")

**Removing Holidays of the same date and merging them with the date.**

In [ ]:
holiday_pd['date']=pd.to_datetime(holiday_pd['date'])
holiday_pd=holiday_pd[holiday_pd['date'] <= "2017-08-15"]
holiday_pd=holiday_pd.drop_duplicates(subset=['date'])
holidays = holiday_pd.groupby(holiday_pd.index).first()
holidays.index = pd.DatetimeIndex(holidays['date'])
holidays=holidays.drop(['date'], axis=1)
df.index = pd.DatetimeIndex(df['date'])
df=df.drop(['date'], axis=1)
df = df.join(holidays, on = 'date', how = 'left')

df1_oil.index = pd.DatetimeIndex(df1_oil['date'])
df1_oil=df1_oil.drop(['date'], axis=1)
df1_oil=df1_oil.drop(['year'], axis=1)
df = df.join(df1_oil, on = 'date', how = 'left')

**Making workday column using holidays and weekends.**

# Feature Engineering


In [ ]:
df['work_day'] = 1
df.loc[df['day'] > 4, 'work_day'] = 0
df.loc[df['description'].notnull(), 'work_day'] = 0
df.loc[df.type == 'Bridge', 'work_day'] = 0
df.loc[df.type == 'Work Day', 'work_day'] = 1
df.loc[df.type == 'Transfer', 'work_day'] = 0
df.loc[(df.type == 'Holiday') & (df.transferred == False), 'work_day'] = 0
df.loc[(df.type == 'Holiday') & (df.transferred == True), 'work_day'] = 0
df.drop(columns = ['locale', 'locale_name', 'description', 'transferred','type'],  inplace = True)
df.info()

In [ ]:
df.head()

**Looking for correlation connected features.**

In [ ]:
corrMatrix = df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

**We saw sales are highest connected with onpromotion.**

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([0,0,1,1])
ax.bar((df.groupby(['day']).mean())['sales'].index,(df.groupby(['day']).mean())['sales'].values)
ax.set(xlabel='Weekdays', ylabel='Mean Prices');
plt.show()

**We found that the highest sales were on weekends.**

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([0,0,1,1])
ax.bar((df.groupby(['month']).mean())['sales'].index,(df.groupby(['month']).mean())['sales'].values)
ax.set(xlabel='Months', ylabel='Mean Prices');
plt.show()

**And the months are increasing more than before.**

**The highest selling month is December.**

# Data pre-processing

In [ ]:
stores.index = stores['store_nbr']
#df1_oil=df1_oil.drop(['date'], axis=1)
#df1_oil=df1_oil.drop(['year'], axis=1)
#df = df.join(stores, on ='store_nbr', how = 'left')
df.join(stores, on='store_nbr', how='left', lsuffix='_left', rsuffix='_right')

In [ ]:
#target
train_df=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train_df.drop(columns = ['id', 'onpromotion'],  inplace = True)
train_df['date']=pd.to_datetime(train_df['date'])
train_y = train_df.set_index(['store_nbr', 'family', 'date']).sort_index()
train_y = train_y.unstack(['store_nbr', 'family'])
train_y

In [ ]:
fr_df= train_df.set_index(['store_nbr', 'family', 'date']).sort_index()

y = fr_df.unstack(['store_nbr', 'family'])

fourier = CalendarFourier(freq='W', order=4)

dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          seasonal=False,
                          additional_terms=[fourier],
                          drop=True)
X = dp.in_sample()
X.head()

In [ ]:
y.head()

In [ ]:
#concate everything
train_df['date']=pd.to_datetime(train_df['date'])
X['day']=X.index.dayofweek
X['month']=X.index.month
X['year']=X.index.year
X.head()

In [ ]:
X = X.join(holidays, on = 'date', how = 'left')
X['work_day'] = 1
X.loc[X['day'] > 4, 'work_day'] = 0
X.loc[X['description'].notnull(), 'work_day'] = 0
X.loc[X.type == 'Bridge', 'work_day'] = 0
X.loc[X.type == 'Work Day', 'work_day'] = 1
X.loc[X.type == 'Transfer', 'work_day'] = 0
X.loc[(X.type == 'Holiday') & (X.transferred == False), 'work_day'] = 0
X.loc[(X.type == 'Holiday') & (X.transferred == True), 'work_day'] = 0
X.drop(columns = ['locale', 'locale_name', 'description', 'transferred','type'],  inplace = True)
X.head()

In [ ]:
X = X.join(df1_oil, on = 'date', how = 'left')
X.head()

In [ ]:
test_df.drop(columns = ['onpromotion'],  inplace = True)
test_df['date']=pd.to_datetime(test_df['date'])
X_test = test_df.set_index(['store_nbr', 'family', 'date']).sort_index()
X_test = X_test.unstack(['store_nbr', 'family'])
X_test = dp.out_of_sample(steps=16,forecast_index=X_test.index)#for last 16 day.
X_test

In [ ]:
test_df = test_df.set_index(['store_nbr', 'family', 'date']).sort_index()
test_df.head()

In [ ]:
X_test['day']=X_test.index.dayofweek
X_test['month']=X_test.index.month
X_test['year']=X_test.index.year
X_test = X_test.join(holidays, on = 'date', how = 'left')
X_test['work_day'] = 1
X_test.loc[X_test['day'] > 4, 'work_day'] = 0
X_test.loc[X_test['description'].notnull(), 'work_day'] = 0
X_test.loc[X_test.type == 'Bridge', 'work_day'] = 0
X_test.loc[X_test.type == 'Work Day', 'work_day'] = 1
X_test.loc[X_test.type == 'Transfer', 'work_day'] = 0
X_test.loc[(X_test.type == 'Holiday') & (X_test.transferred == False), 'work_day'] = 0
X_test.loc[(X_test.type == 'Holiday') & (X_test.transferred == True), 'work_day'] = 0
X_test.drop(columns = ['locale', 'locale_name', 'description', 'transferred','type'],  inplace = True)
X_test.head()

In [ ]:
X_test = X_test.join(df1_oil, on = 'date', how = 'left')
X_test.head()

# Model Training

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True)
model.fit(X, y)

# Submission

In [ ]:
y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(test_df.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)